# models for opcode frequency

## Import Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import time
import json
import pandas as pd
from pathlib import Path
import joblib

import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier


## Const

In [3]:
PATH = Path.cwd().parents[1]
DATA_PATH = os.path.join(PATH, 'data/processed')
MODEL_PATH = os.path.join(PATH, 'models')

In [4]:
with open(os.path.join(DATA_PATH, 'feature-opcode-freq_list.json')) as f:
    feature_list = json.load(f)

with open(os.path.join(DATA_PATH, 'labels-opcode-freq.json')) as f:
    labels = json.load(f)

In [5]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train-opcode-freq.csv'))
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test-opcode-freq.csv'))

X_train = train_df[feature_list]
y_train = train_df[labels]

X_test = test_df[feature_list]
y_test = test_df[labels]

In [6]:
X_train.head()

,PUSH1,MSTORE,CALLDATASIZE,LT,PUSH2,JUMPI,CALLDATALOAD,PUSH29,SWAP1,DIV,...,UNKNOWN_0xc6,UNKNOWN_0xe1,INVALID_0x70,PUSH30,DUP16,UNKNOWN_0x2b,UNKNOWN_0xd8,INVALID_0x7a,UNKNOWN_0xf9,INVALID_0x7f
0,1,0,2,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,16,2,2,0,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,40,5,5,2,32,10,2,0,7,0,...,0,0,0,0,1,1,0,0,0,0
3,53,7,5,2,43,9,2,0,8,0,...,0,0,0,0,0,0,0,0,0,0
4,178,32,7,7,134,41,6,0,66,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_train.head()

,mint,leak,limit
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0


## Traditional Machine Learning Models

In [8]:
models = {
    "Logistic Regression": MultiOutputClassifier(LogisticRegression(max_iter=1000)),
    "Random Forest": MultiOutputClassifier(RandomForestClassifier()),
    "Gradient Boosting": MultiOutputClassifier(GradientBoostingClassifier()),
    "AdaBoost": MultiOutputClassifier(AdaBoostClassifier()),
    "SVM (Linear)": MultiOutputClassifier(SVC(kernel="linear")),
    "KNN": MultiOutputClassifier(KNeighborsClassifier()),
    "Naive Bayes": MultiOutputClassifier(GaussianNB()),
    "MLP Classifier": MultiOutputClassifier(MLPClassifier(max_iter=300)),
    "XGBoost": MultiOutputClassifier(XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    "LightGBM": MultiOutputClassifier(LGBMClassifier()),
    "DecisionTree": MultiOutputClassifier(DecisionTreeClassifier())
}

In [9]:
results = []

for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()

    y_pred = model.predict(X_test)

    results.append({
        "Classifier": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred, average="macro"),  # change to 'macro' if multi-class
        "Recall": recall_score(y_test, y_pred, average="macro"),
        "F1-Score": f1_score(y_test, y_pred, average="macro"),
        "Training Time": round(end - start, 3)
    })

[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000629 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1889
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 94
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

### Result

In [10]:
df = pd.DataFrame(results)
df.sort_values(by="F1-Score", ascending=False, inplace=True)
df

,Classifier,Accuracy,Precision,Recall,F1-Score,Training Time
9,LightGBM,0.428571,0.888889,0.388889,0.484127,0.118
5,KNN,0.285714,0.571429,0.407407,0.475000,0.010
8,XGBoost,0.428571,0.805556,0.388889,0.471306,0.803
10,DecisionTree,0.285714,0.543651,0.407407,0.451720,0.025
1,Random Forest,0.357143,0.888889,0.351852,0.438672,0.502
0,Logistic Regression,0.285714,0.527778,0.351852,0.422222,0.638
3,AdaBoost,0.285714,0.477778,0.333333,0.387302,0.486
2,Gradient Boosting,0.285714,0.694444,0.277778,0.360195,1.115
4,SVM (Linear),0.285714,0.416667,0.277778,0.318627,0.162
7,MLP Classifier,0.285714,0.500000,0.166667,0.250000,0.935


### Tuning

In [11]:
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 500),
        "learning_rate": trial.suggest_float("learning_rate", 1e-6, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "num_leaves": trial.suggest_int("num_leaves", 3, 255),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1
    }

    model = MultiOutputClassifier(LGBMClassifier(**params))

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average="macro", zero_division=0)

In [12]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Tuned LGBMClassifier (MultiOutput):")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average="macro", zero_division=0))
print("Recall:", recall_score(y_test, y_pred, average="macro", zero_division=0))
print("F1 Score:", f1_score(y_test, y_pred, average="macro", zero_division=0))

[I 2025-07-12 00:42:42,289] A new study created in memory with name: no-name-67d1d369-dfda-4549-9b7f-365441a4f4b4


[LightGBM] [Info] Number of positive: 39, number of negative: 98
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000682 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1774
[LightGBM] [Info] Number of data points in the train set: 137, number of used features: 73
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.284672 -> initscore=-0.921406
[LightGBM] [Info] Start training from score -0.921406
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

[I 2025-07-12 00:42:42,673] Trial 0 finished with value: 0.47130647130647124 and parameters: {'n_estimators': 273, 'learning_rate': 0.09756834108429244, 'max_depth': 13, 'num_leaves': 115, 'min_child_samples': 56, 'subsample': 0.5984437562683029, 'colsample_bytree': 0.9280193353737213, 'reg_alpha': 0.0011656724780834467, 'reg_lambda': 4.412565141674141e-06}. Best is trial 0 with value: 0.47130647130647124.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-12 00:42:43,066] Trial 1 finished with value: 0.32857142857142857 and parameters: {'n_estimators': 244, 'learning_rate': 0.007764494565889812, 'max_depth': 11, 'num_leaves': 212, 'min_child_samples': 37, 'subsample': 0.5736148345970538, 'colsample_bytree': 0.8561982139071842, 'reg_alpha': 0.37508722878971706, 'reg_lambda': 2.664736591317692}. Best is trial 0 with value: 0.47130647130647124.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-12 00:42:58,625] Trial 48 finished with value: 0.0 and parameters: {'n_estimators': 135, 'learning_rate': 0.0003079639351374397, 'max_depth': 9, 'num_leaves': 85, 'min_child_samples': 3, 'subsample': 0.7284580157695014, 'colsample_bytree': 0.9672213289010065, 'reg_alpha': 0.00016249259277798412, 'reg_lambda': 2.004093073257318e-05}. Best is trial 12 with value: 0.5793650793650793.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-07-12 00:42:59,321] Trial 49 finished with value: 0.48412698412698413 and parameters: {'n_estimators': 168, 'learning_rate': 0.032431235465729076, 'max_depth': 8, 'num_leaves': 107, 'min_child_samples': 22, 'subsample': 0.6659662101978585, 'colsample_bytree': 0.8923887009413356, 'reg_alpha': 0.0033086021553518498, 'reg_lambda': 0.00023693893101832937}. Best is trial 12 with value: 0.5793650793650793.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [13]:
joblib.dump(model, os.path.join(MODEL_PATH, f'best_lgbm-moc_model_on_crpwarner_opcode_freq.pkl'))

['/Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/models/best_lgbm-moc_model_on_crpwarner_opcode_freq.pkl']

### K-Fold (K=3)

In [14]:
NUM_FOLDS = 3
results = []
best_model = MultiOutputClassifier(LGBMClassifier(**study.best_params, random_state=42))
best_f1 = 0
best_fold = 0

In [15]:
for fold in range(NUM_FOLDS):
    print(f"=========== Fold-{fold} ===========")
    train_path = os.path.join(DATA_PATH, f'train_fold_{fold}-opcode-freq.csv')
    val_path = os.path.join(DATA_PATH, f'val_fold_{fold}-opcode-freq.csv')

    train_df = pd.read_csv(train_path)
    val_df   = pd.read_csv(val_path)

    X_train = train_df[feature_list]
    y_train = train_df[labels]

    X_val = val_df[feature_list]
    y_val = val_df[labels]

    # Train model
    model = MultiOutputClassifier(LGBMClassifier(**study.best_params, random_state=42))
    model.fit(X_train, y_train)

    # Evaluate
    y_pred = model.predict(X_val)
    report = classification_report(y_val, y_pred, target_names=labels, output_dict=True)
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average="macro", zero_division=0)

    results.append({'fold': fold, 'accuracy': acc, 'report': report})
    print(f"Accuracy: {acc}")
    print("Precision:", precision_score(y_val, y_pred, average="macro", zero_division=0))
    print("Recall:", recall_score(y_val, y_pred, average="macro", zero_division=0))
    print("F1 Score:", f1)

    if best_f1 < f1:
        best_model = model
        best_fold = fold
## Step 6: Average Performance Summary
print("\n===== Overall Summary =====")
avg_acc = sum([r['accuracy'] for r in results]) / NUM_FOLDS
print(f"Average Accuracy: {avg_acc:.4f}")

# Save model
joblib.dump(best_model, os.path.join(MODEL_PATH, f'best_ada-ovr_model_on_crpwarner_opcode_freq_from_fold{best_fold}.pkl'))

=========== Fold-0 ===========
[LightGBM] [Info] Number of positive: 13, number of negative: 33
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000414 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1044
[LightGBM] [Info] Number of data points in the train set: 46, number of used features: 152
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.282609 -> initscore=-0.931558
[LightGBM] [Info] Start training from score -0.931558
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

['/Users/napatcholthaipanich/Dev/master/dissertation/workspace/ml/models/best_ada-ovr_model_on_crpwarner_opcode_freq_from_fold2.pkl']